In [168]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

In [169]:
dico = {3 : "points", 9 : "rebonds", 10 : "assists", 11 : "steals", 13 : "blocks", 16 : "fouls drown",\
        4 : "missed fg1", 5 : "missed fg2", 6 : "missed ft", 12 : "turnovers", 14 : "shots rejected", 15 : "fouls committed", \
       17 : "PIR"}
results = requests.get("http://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2017&cat=Valuation&agg=Accumulated")
soup = BeautifulSoup(results.text, 'html.parser')
df = pd.DataFrame(columns=dico.values())

In [170]:
dico.values()

dict_values(['points', 'missed fg1', 'missed fg2', 'missed ft', 'rebonds', 'assists', 'steals', 'turnovers', 'blocks', 'shots rejected', 'fouls committed', 'fouls drown', 'PIR'])

In [171]:
i = 0
for tr in soup.find("table").findAll("tr")[1:]:
    res = requests.get("http://www.euroleague.net/"+tr.find_all("td")[1].find('a', href=True)["href"])
    player = BeautifulSoup(res.text, 'html.parser')
    for line in (player.findAll("table")[1].findAll("tr")):
        td = line.findAll('td')
        try : 
            for key in dico.keys():
                df.loc[i, dico[key]] = td[key].text
            i += 1
            print(i)
        except : 
            pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


KeyboardInterrupt: 

In [172]:
df = df[~df["missed fg2"].str.contains("%")]

In [173]:
df.replace({"\n" : "", "%" : "", "" : "0", '\xa0' : "0"}, regex=True, inplace=True)

In [174]:
def parse_str(x):
    temp = x.split("/")
    if len(temp) == 2:
        return int(temp[1]) - int(temp[0])
    else : 
        return(x)
    
df["missed fg1"] = df["missed fg1"].map(lambda x : parse_str(x))
df["missed fg2"] = df["missed fg2"].map(lambda x : parse_str(x))
df["missed ft"] = df["missed ft"].map(lambda x : parse_str(x))


In [175]:
df = df.astype(float)

## 1st idea :
solve a linar equation

In [189]:
a = df.drop("PIR", axis=1).iloc[:df.shape[1]-1, :]

In [190]:
b = df["PIR"][:df.shape[1]-1]

In [192]:
np.linalg.solve(a, b)

array([ 1.87630014, -0.62460474,  0.60326552, -7.2301772 ,  1.00286379,
       -3.97191894,  1.55191615, -1.59676233,  7.99164728, -2.09897977,
       -4.92076845,  1.33333333])

In [ ]:
np.

In [177]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(fit_intercept=False)
lr.fit(df.drop("PIR", axis=1), df["PIR"])

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [154]:
lr.coef_


array([ 1., -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.])

In [167]:
import numpy as np 
np.round(np.dot(df.drop("PIR", axis=1), lr.coef_), 1) == df["PIR"].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,